# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [1]:
from google.colab import drive
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#importando as bases
X_train = pd.read_csv('/content/drive/MyDrive/Ebac/Profissão: Cientista de dados/Ciência de dados/Módulo 17/Ex 1/X_train_new.csv')
X_test = pd.read_csv('/content/drive/MyDrive/Ebac/Profissão: Cientista de dados/Ciência de dados/Módulo 17/Ex 1/X_test_new.csv')
y_train = pd.read_csv('/content/drive/MyDrive/Ebac/Profissão: Cientista de dados/Ciência de dados/Módulo 17/Ex 1/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt', header=None, names=['action'])
y_test = pd.read_csv('/content/drive/MyDrive/Ebac/Profissão: Cientista de dados/Ciência de dados/Módulo 17/Ex 1/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt', header=None, names=['action'])

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [4]:
# separando 25% da base de treino como validação
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.25, random_state=42)

In [5]:
X_train.columns.nunique()

562

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [6]:
%%time
#gerando a árvore de decisão
dt_clf = DecisionTreeClassifier(max_depth=4, random_state=42)
dt_clf.fit(X_train, y_train)

pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(f'Acurácia da DecisionTreeClassifier : {accuracy:.4f} \n')

Acurácia da DecisionTreeClassifier : 0.8344 

CPU times: user 1.52 s, sys: 16.8 ms, total: 1.54 s
Wall time: 1.56 s


In [7]:
# achando as 3 variaveis com maior importancia
importances = dt_clf.feature_importances_
forest_importances = pd.Series(importances)
melhores = np.partition(forest_importances, -3)[-3:]
melhores

array([0.16460791, 0.25409067, 0.28530319])

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [8]:
%%time

# Selecionar apenas os três melhores recursos
dados_selecionados = X_train.iloc[:, melhores]

# Criar uma nova árvore de decisão com base nos três melhores recursos
nova_arvore = DecisionTreeClassifier(random_state=42, max_depth=4)
nova_arvore.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
previsoes = nova_arvore.predict(X_test)

CPU times: user 1.6 s, sys: 12.2 ms, total: 1.61 s
Wall time: 1.67 s


In [13]:
clf = DecisionTreeClassifier(max_depth=4,
                             random_state=42).fit(X=X_train,
                                                  y=y_train)

three_best_vars = (pd.DataFrame(data={'var': X_train.columns,
                                      'feature_importances_': clf.feature_importances_})
                      .sort_values(by='feature_importances_',
                                  ascending=False))

In [15]:
print(three_best_vars)

cols = three_best_vars['var'].values[:3]
print('3 melhores variáveis:', cols)

                           var  feature_importances_
53         tGravityAcc-min()-X              0.285303
272           fBodyAcc-mad()-X              0.254091
42        tGravityAcc-mean()-Y              0.164608
202          tBodyAccMag-std()              0.140142
70   tGravityAcc-arCoeff()-Y,1              0.122278
..                         ...                   ...
184  tBodyGyroJerk-entropy()-Y              0.000000
183  tBodyGyroJerk-entropy()-X              0.000000
182      tBodyGyroJerk-iqr()-Z              0.000000
181      tBodyGyroJerk-iqr()-Y              0.000000
561       angle(Z,gravityMean)              0.000000

[562 rows x 2 columns]
3 melhores variáveis: ['tGravityAcc-min()-X' 'fBodyAcc-mad()-X' 'tGravityAcc-mean()-Y']


### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [9]:
# Avaliar a precisão do modelo
acuracia = accuracy_score(y_test, previsoes)
print(f'Acurácia da DecisionTreeClassifier : {acuracia:.4f} \n')

Acurácia da DecisionTreeClassifier : 0.8344 



Podemos verificar então que o modelo utilizando as melhores variáveis teve uma acurácia igual à da primeira árvore, porém, tivemos um ganho de performance